In [1]:
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
import matplotlib
font = {'family' : 'MicroSoft YaHei',
        'weight' : 'bold',
        'size' : 10}
matplotlib.rc('font', **font)

## 导入数据

In [2]:
file1 = r"D:\系统默认\Documents\gradProject\rawData\data1\order_grid.csv"
file2 = r"D:\系统默认\Documents\gradProject\rawData\data1\POI_grid_count1.csv"
file3 = r"D:\系统默认\Documents\gradProject\rawData\data1\weather.csv"

df_order = pd.read_csv(file1)
df_poi = pd.read_csv(file2)
df_weather = pd.read_csv(file3)

In [5]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11125061 entries, 0 to 11125060
Data columns (total 13 columns):
 #   Column         Dtype  
---  ------         -----  
 0   OrderID        int64  
 1   VehicleType    int64  
 2   Appointment    int64  
 3   Distance       int64  
 4   ArrivalTime    object 
 5   DepartureTime  object 
 6   Cost           float64
 7   TravelTime     float64
 8   lon_e          float64
 9   lat_e          float64
 10  lon_s          float64
 11  lat_s          float64
 12  GridID         float64
dtypes: float64(7), int64(4), object(2)
memory usage: 1.1+ GB


In [6]:
df_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489 entries, 0 to 488
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   GridID       489 non-null    int64
 1   poi_count_0  489 non-null    int64
 2   poi_count_1  489 non-null    int64
 3   poi_count_2  489 non-null    int64
 4   poi_count_3  489 non-null    int64
 5   poi_count_4  489 non-null    int64
 6   total_count  489 non-null    int64
dtypes: int64(7)
memory usage: 26.9 KB


In [7]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     184 non-null    object 
 1   week     184 non-null    int64  
 2   temp_h   184 non-null    float64
 3   temp_l   184 non-null    float64
 4   weather  184 non-null    int64  
 5   wind     184 non-null    float64
 6   workday  184 non-null    int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 10.2+ KB


## 函数

In [59]:
def basic_info(column):
    max = column.max()
    min = column.min()
    mean = column.mean()
    std = column.std()
    return max, min, mean, std

In [60]:
def spilt_quantile(column, num):
    if num == 3:
        quantiles = column.quantile([0.333,0.666])
        # if column_value > quantiles[0.666]:
        #     return 'High'
        # elif column_value > quantiles[0.333]:
        #     return 'Medium'
        # else:
        #     return 'Low'
    if num == 4:
        quantiles = column.quantile([0.25, 0.5, 0.75])
    if num ==5:
        quantiles = column.quantile([0.2, 0.4, 0.6, 0.8])
    return quantiles

In [61]:
def demand_level3(column_value, quantiles):
    if column_value > quantiles[0.666]:
        return 'High'
    elif column_value > quantiles[0.333]:
        return 'Medium'
    else:
        return 'Low'

## Input for BN

### 1. 时间切片 每10min/1h为一个片段

In [3]:
df_order['DepartureTime'] = pd.to_datetime(df_order['DepartureTime'])

# 定义时间片段的长度（每十分钟一个片段）
segment_length = 60 * 10  # 10分钟，单位为秒
segment_length1 = 60 * 60 # 1小时，单位为秒

# 计算时间片段的 ID
df_order['TSegmentID'] = ((df_order['DepartureTime'].dt.hour * 3600 + df_order['DepartureTime'].dt.minute * 60 + df_order['DepartureTime'].dt.second) // segment_length).astype(int)
df_order['hour'] = ((df_order['DepartureTime'].dt.hour * 3600 + df_order['DepartureTime'].dt.minute * 60 + df_order['DepartureTime'].dt.second) // segment_length1).astype(int)

# 检查时间片段 ID 是否超过总片段数（144个）
df_order['TSegmentID'] = df_order['TSegmentID'] % 144
df_order['hour'] = df_order['hour'] % 24


In [6]:
from datetime import datetime, timedelta

df_order['DepartureTime'] = pd.to_datetime(df_order['DepartureTime'])  # 将DepartureTime列转换为datetime类型

# 定义函数将时间映射到下一个10分钟间隔
def map_to_next_10_minutes(dt):
    next_time = datetime(dt.year, dt.month, dt.day, dt.hour, dt.minute // 10 * 10) + timedelta(minutes=10)
    return next_time

# 对DataFrame中的DepartureTime列进行映射处理
df_order['MappedTime'] = df_order['DepartureTime'].apply(map_to_next_10_minutes)

In [7]:
df_order

,OrderID,VehicleType,Appointment,Distance,ArrivalTime,DepartureTime,Cost,TravelTime,lon_e,lat_e,lon_s,lat_s,GridID,TSegmentID,hour,MappedTime
0,17592364494589,1,0,35315,2017-04-30 23:52:17,2017-04-30 23:03:40,171.0,49.0,110.2152,20.0518,110.4662,19.9375,362.0,138,23,2017-04-30 23:10:00
1,17592364709509,3,0,20297,2017-05-01 00:03:00,2017-04-30 23:14:37,66.0,49.0,110.4751,19.9419,110.3484,20.0216,448.0,139,23,2017-04-30 23:20:00
2,17592364964467,3,0,8479,2017-04-30 23:54:23,2017-04-30 23:20:51,25.0,34.0,110.3399,20.0243,110.3633,19.9885,405.0,140,23,2017-04-30 23:30:00
3,17592364820364,3,0,16404,2017-04-30 23:46:11,2017-04-30 23:21:09,52.0,25.0,110.3258,20.0293,110.2010,20.0095,421.0,140,23,2017-04-30 23:30:00
4,17592364821087,3,0,6151,2017-04-30 23:56:16,2017-04-30 23:21:18,19.0,35.0,110.3318,20.0110,110.3566,20.0388,468.0,140,23,2017-04-30 23:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11125056,35184675948147,3,0,4150,2017-10-31 23:54:13,2017-10-31 23:47:58,13.0,7.0,110.3261,20.0247,110.3442,20.0498,467.0,142,23,2017-10-31 23:50:00
11125057,35184675882692,3,0,2193,2017-10-31 23:54:08,2017-10-31 23:48:55,9.5,6.0,110.3204,19.9939,110.3258,20.0059,426.0,142,23,2017-10-31 23:50:00
11125058,35184675882963,3,0,3132,2017-10-31 23:55:59,2017-10-31 23:49:11,9.8,6.0,110.3367,20.0300,110.3195,20.0325,447.0,142,23,2017-10-31 23:50:00
11125059,35184675949473,3,0,2985,2017-10-31 23:55:13,2017-10-31 23:49:20,9.5,6.0,110.3407,20.0538,110.3243,20.0670,480.0,142,23,2017-10-31 23:50:00


### 2. 计算每一天每个区块内每个时间序列需求的数量

In [8]:
# 提取日期、月份信息
df_order['date'] = df_order['DepartureTime'].dt.date
df_order['month'] = df_order['DepartureTime'].dt.month
df_order['day'] = df_order['DepartureTime'].dt.day

# 计算每一天每个区块内每个时间序列记录的数目
# demand_min = df_order.groupby(['GridID', 'TSegmentID', 'month', 'date']).size().reset_index(name='Demand')
demand_h = df_order.groupby(['GridID', 'hour', 'month', 'date']).size().reset_index(name='Demand')
# demand_tg = df_order.groupby(['GridID', 'MappedTime']).size().reset_index(name='Demand')

# 按照 Demand 列从大到小排序
# demand_min = demand_min.sort_values(by='Demand', ascending=False)
demand_h = demand_h.sort_values(by='Demand', ascending=False)

# 打印结果
# demand_min
# demand_tg

In [9]:
demand_h

,GridID,hour,month,date,Demand
182557,448.0,17,7,2017-07-22,866
182613,448.0,17,9,2017-09-16,824
183196,448.0,20,10,2017-10-17,814
183094,448.0,20,7,2017-07-07,794
182593,448.0,17,8,2017-08-27,791
...,...,...,...,...,...
104418,409.0,13,5,2017-05-01,1
104412,409.0,12,10,2017-10-26,1
104409,409.0,12,10,2017-10-23,1
104408,409.0,12,10,2017-10-22,1


In [130]:
# 导出部分示例数据
test = demand_min.head(10)
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\demand_min_test.csv'
test.to_csv(csv_save, index=False)

### 3. 加入天气数据

In [10]:
# 日期数据类型转化
df_weather['date'] = pd.to_datetime(df_weather['date'])
# demand_min['date'] = pd.to_datetime(demand_min['date'])
demand_h['date'] = pd.to_datetime(demand_h['date'])

In [11]:
# 连接demand和weather
# demand_min = demand_min.set_index('date').join(df_weather.set_index('date'),how='inner')
demand_h = demand_h.set_index('date').join(df_weather.set_index('date'),how='inner')
demand_h

,GridID,hour,month,Demand,week,temp_h,temp_l,weather,wind,workday
date,,,,,,,,,,
2017-07-22,448.0,17,7,866,6,29.0,25.0,4,2.0,0
2017-09-16,448.0,17,9,824,6,32.0,26.0,1,3.0,0
2017-10-17,448.0,20,10,814,2,27.0,24.0,4,2.0,1
2017-07-07,448.0,20,7,794,5,31.0,25.0,3,3.0,1
2017-08-27,448.0,17,8,791,7,30.0,26.0,4,3.0,0
...,...,...,...,...,...,...,...,...,...,...
2017-05-01,409.0,13,5,1,1,32.0,25.0,1,5.0,1
2017-10-26,409.0,12,10,1,4,21.0,20.4,1,3.5,1
2017-10-23,409.0,12,10,1,1,27.0,21.0,1,2.0,1


In [12]:
# 将索引转换为普通的一列
# demand_min = demand_min.reset_index()
demand_h = demand_h.reset_index()

### 4. 连接demand和POI

In [13]:
# 连接demand和POI
# demand_min = demand_min.set_index('GridID').join(df_poi.set_index('GridID'),how='inner')
demand_h = demand_h.set_index('GridID').join(df_poi.set_index('GridID'),how='inner')

# 将索引转换为普通的一列
# demand_min = demand_min.reset_index()
demand_h = demand_h.reset_index()

In [14]:
demand_h

,GridID,date,hour,month,Demand,week,temp_h,temp_l,weather,wind,workday,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count
0,448.0,2017-07-22,17,7,866,6,29.0,25.0,4,2.0,0,742,655,6571,1773,2758,12499
1,448.0,2017-09-16,17,9,824,6,32.0,26.0,1,3.0,0,742,655,6571,1773,2758,12499
2,448.0,2017-10-17,20,10,814,2,27.0,24.0,4,2.0,1,742,655,6571,1773,2758,12499
3,448.0,2017-07-07,20,7,794,5,31.0,25.0,3,3.0,1,742,655,6571,1773,2758,12499
4,448.0,2017-08-27,17,8,791,7,30.0,26.0,4,3.0,0,742,655,6571,1773,2758,12499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245198,409.0,2017-05-01,13,5,1,1,32.0,25.0,1,5.0,1,6,31,49,50,26,162
245199,409.0,2017-10-26,12,10,1,4,21.0,20.4,1,3.5,1,6,31,49,50,26,162
245200,409.0,2017-10-23,12,10,1,1,27.0,21.0,1,2.0,1,6,31,49,50,26,162
245201,409.0,2017-10-22,12,10,1,7,27.0,22.0,1,2.0,0,6,31,49,50,26,162


In [137]:
# 保存数据
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\demand_min.csv'
demand_min.to_csv(csv_save, index=False)

### 5. 去除订单数量过少的区域（平均每日<10单）

In [16]:
# 确定数据集包含的天数范围
first_date = pd.to_datetime(demand_h['date'].min())
last_date = pd.to_datetime(demand_h['date'].max())
total_days = (last_date - first_date).days + 1
first_date, last_date, total_days

(Timestamp('2017-05-01 00:00:00'), Timestamp('2017-10-31 00:00:00'), 184)

In [23]:
# 计算每个地区的总 Demand
grid_total_demand = demand_h.groupby('GridID')['Demand'].sum().reset_index()

# 计算每个地区的总 Demand，然后除以总天数
grid_avg_demand = grid_total_demand.copy()
grid_avg_demand['Demand'] /= total_days

# 找到平均每天 Demand 小于 10 的区域
low_demand_grid = grid_avg_demand[grid_avg_demand['Demand'] < 10]['GridID']

# 3. 将这些区域从原表格中排除
demand_h = demand_h[~demand_h['GridID'].isin(low_demand_grid)]

In [20]:
filtered_demand_h.nunique()

GridID          66
date           184
hour            24
month            6
Demand         714
week             7
temp_h          14
temp_l          11
weather          5
wind             9
workday          2
poi_count_0     43
poi_count_1     43
poi_count_2     61
poi_count_3     53
poi_count_4     50
total_count     62
dtype: int64

In [24]:
demand_h.nunique()

GridID          66
date           184
hour            24
month            6
Demand         714
week             7
temp_h          14
temp_l          11
weather          5
wind             9
workday          2
poi_count_0     43
poi_count_1     43
poi_count_2     61
poi_count_3     53
poi_count_4     50
total_count     62
dtype: int64

# 数据离散化处理

In [25]:
# 删除部分贝叶斯网络中不需要的数据
demand_h = demand_h.drop(columns=['GridID','date','month','week'])
demand_h

,hour,Demand,temp_h,temp_l,weather,wind,workday,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count
0,17,866,29.0,25.0,4,2.0,0,742,655,6571,1773,2758,12499
1,17,824,32.0,26.0,1,3.0,0,742,655,6571,1773,2758,12499
2,20,814,27.0,24.0,4,2.0,1,742,655,6571,1773,2758,12499
3,20,794,31.0,25.0,3,3.0,1,742,655,6571,1773,2758,12499
4,17,791,30.0,26.0,4,3.0,0,742,655,6571,1773,2758,12499
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245197,13,1,30.0,24.0,2,2.0,1,6,31,49,50,26,162
245198,13,1,32.0,25.0,1,5.0,1,6,31,49,50,26,162
245199,12,1,21.0,20.4,1,3.5,1,6,31,49,50,26,162
245200,12,1,27.0,21.0,1,2.0,1,6,31,49,50,26,162


### 1. 时间映射

In [26]:
hour_map={      #    0-3凌晨   4-7早晨   8-11 上午    12-15下午  16-19傍晚    20-23夜晚    
    0:'wee',
    1:'wee',
    2:'wee',
    3:'wee',
    4:'morn',
    5:'morn',
    6:'morn',
    7:'morn',
    8:'forenoon',
    9:'forenoon',
    10:'forenoon',
    11:'forenoon',
    12:'afternoon',
    13:'afternoon',
    14:'afternoon',
    15:'afternoon',
    16:'dusk',
    17:'dusk',
    18:'dusk',
    19:'dusk',
    20:'night',
    21:'night',
    22:'night',
    23:'night'}
demand_h['hour'] = demand_h['hour'].map(hour_map)
demand_h

,hour,Demand,temp_h,temp_l,weather,wind,workday,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count
0,dusk,866,29.0,25.0,4,2.0,0,742,655,6571,1773,2758,12499
1,dusk,824,32.0,26.0,1,3.0,0,742,655,6571,1773,2758,12499
2,night,814,27.0,24.0,4,2.0,1,742,655,6571,1773,2758,12499
3,night,794,31.0,25.0,3,3.0,1,742,655,6571,1773,2758,12499
4,dusk,791,30.0,26.0,4,3.0,0,742,655,6571,1773,2758,12499
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245197,afternoon,1,30.0,24.0,2,2.0,1,6,31,49,50,26,162
245198,afternoon,1,32.0,25.0,1,5.0,1,6,31,49,50,26,162
245199,afternoon,1,21.0,20.4,1,3.5,1,6,31,49,50,26,162
245200,afternoon,1,27.0,21.0,1,2.0,1,6,31,49,50,26,162


### 分类型变量映射

In [27]:
workday_map={      #    0否  1是  
    0:'n',
    1:'y'}
demand_h['workday'] = demand_h['workday'].map(workday_map)

weather_map = {
    1: 'best',
    2: 'better',
    3: 'medium',
    4: 'worse',
    5: 'worst',
}
demand_h['weather'] = demand_h['weather'].map(weather_map)
demand_h

,hour,Demand,temp_h,temp_l,weather,wind,workday,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count
0,dusk,866,29.0,25.0,worse,2.0,n,742,655,6571,1773,2758,12499
1,dusk,824,32.0,26.0,best,3.0,n,742,655,6571,1773,2758,12499
2,night,814,27.0,24.0,worse,2.0,y,742,655,6571,1773,2758,12499
3,night,794,31.0,25.0,medium,3.0,y,742,655,6571,1773,2758,12499
4,dusk,791,30.0,26.0,worse,3.0,n,742,655,6571,1773,2758,12499
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245197,afternoon,1,30.0,24.0,better,2.0,y,6,31,49,50,26,162
245198,afternoon,1,32.0,25.0,best,5.0,y,6,31,49,50,26,162
245199,afternoon,1,21.0,20.4,best,3.5,y,6,31,49,50,26,162
245200,afternoon,1,27.0,21.0,best,2.0,y,6,31,49,50,26,162


### 数值型变量投影

In [28]:
columns = ['Demand','temp_h', 'temp_l','wind','poi_count_0','poi_count_1','poi_count_2','poi_count_3','poi_count_4','total_count']
for column in columns:
     # 计算最小值和最大值
    min_value = demand_h[column].min()
    max_value = demand_h[column].max()
    
    # 进行 Min-Max 归一化,并替换
    demand_h[column] = (demand_h[column] - min_value) / (max_value - min_value)
demand_h

,hour,Demand,temp_h,temp_l,weather,wind,workday,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count
0,dusk,1.000000,0.5000,0.666667,worse,0.333333,n,1.000000,1.000000,1.000000,0.727422,1.000000,1.000000
1,dusk,0.951445,0.6875,0.777778,best,0.500000,n,1.000000,1.000000,1.000000,0.727422,1.000000,1.000000
2,night,0.939884,0.3750,0.555556,worse,0.333333,y,1.000000,1.000000,1.000000,0.727422,1.000000,1.000000
3,night,0.916763,0.6250,0.666667,medium,0.500000,y,1.000000,1.000000,1.000000,0.727422,1.000000,1.000000
4,dusk,0.913295,0.5625,0.777778,worse,0.500000,n,1.000000,1.000000,1.000000,0.727422,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245197,afternoon,0.000000,0.5625,0.555556,better,0.333333,y,0.008086,0.047328,0.007306,0.020115,0.009068,0.012092
245198,afternoon,0.000000,0.6875,0.666667,best,0.833333,y,0.008086,0.047328,0.007306,0.020115,0.009068,0.012092
245199,afternoon,0.000000,0.0000,0.155556,best,0.583333,y,0.008086,0.047328,0.007306,0.020115,0.009068,0.012092
245200,afternoon,0.000000,0.3750,0.222222,best,0.333333,y,0.008086,0.047328,0.007306,0.020115,0.009068,0.012092


In [29]:
def demand_level3(demand):
    if demand > 0.6666666:
        return 'High'
    elif demand > 0.3333333:
        return 'Medium'
    else:
        return 'Low'

for column in columns:
    demand_h[column] = demand_h[column].apply(demand_level3)

demand_h

,hour,Demand,temp_h,temp_l,weather,wind,workday,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count
0,dusk,High,Medium,High,worse,Medium,n,High,High,High,High,High,High
1,dusk,High,High,High,best,Medium,n,High,High,High,High,High,High
2,night,High,Medium,Medium,worse,Medium,y,High,High,High,High,High,High
3,night,High,Medium,High,medium,Medium,y,High,High,High,High,High,High
4,dusk,High,Medium,High,worse,Medium,n,High,High,High,High,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245197,afternoon,Low,Medium,Medium,better,Medium,y,Low,Low,Low,Low,Low,Low
245198,afternoon,Low,High,High,best,High,y,Low,Low,Low,Low,Low,Low
245199,afternoon,Low,Low,Low,best,Medium,y,Low,Low,Low,Low,Low,Low
245200,afternoon,Low,Medium,Low,best,Medium,y,Low,Low,Low,Low,Low,Low


In [30]:
# 保存数据
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\for_BN_short.csv'
demand_h.to_csv(csv_save, index=False)

# 缩减用于GAN的数据集

In [146]:
demand_min

,GridID,date,TSegmentID,month,Demand,week,temp_h,temp_l,weather,wind,workday,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count
0,448.0,2017-07-19,105,7,172,3,32.0,25.0,2,2.0,1,742,655,6571,1773,2758,12499
1,448.0,2017-08-27,103,8,166,7,30.0,26.0,4,3.0,0,742,655,6571,1773,2758,12499
2,448.0,2017-08-03,104,8,164,4,34.0,27.0,2,3.0,1,742,655,6571,1773,2758,12499
3,448.0,2017-07-07,123,7,161,5,31.0,25.0,3,3.0,1,742,655,6571,1773,2758,12499
4,448.0,2017-10-21,106,10,160,6,27.0,23.0,1,2.0,0,742,655,6571,1773,2758,12499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975906,412.0,2017-05-19,36,5,1,5,29.0,24.0,2,4.0,1,3,7,266,39,45,360
975907,412.0,2017-05-17,36,5,1,3,30.0,24.0,1,5.0,1,3,7,266,39,45,360
975908,412.0,2017-05-12,36,5,1,5,32.0,25.0,2,4.0,1,3,7,266,39,45,360
975909,412.0,2017-05-09,36,5,1,2,31.0,25.0,2,3.0,1,3,7,266,39,45,360


In [160]:
# 筛选7.1-7.21三周的数据作为训练集
start_date = '2017-07-01'
end_date = '2017-07-21'

# 使用条件筛选选择日期在指定范围内的数据
training_demand = demand_min[(demand_min['date'] >= start_date) & (demand_min['date'] <= end_date)]

# # 保留部分值
# # 假设要保存的列名为 column1 和 column2
# columns_to_save = ['Demand', 'TSegmentID','week','weather','total_count']

# # 使用 loc 方法提取指定列的数据并创建新的 DataFrame
# training_demand = training_demand.loc[:, columns_to_save]

# training_demand

In [151]:
# 保存数据
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\demand_training.csv'
training_demand.to_csv(csv_save, index=False)

In [158]:
pd.get_dummies??

Signature:
pd.get_dummies(
    data,
    prefix=None,
    prefix_sep: 'str | Iterable[str] | dict[str, str]' = '_',
    dummy_na: 'bool' = False,
    columns=None,
    sparse: 'bool' = False,
    drop_first: 'bool' = False,
    dtype: 'NpDtype | None' = None,
) -> 'DataFrame'
Source:   
def get_dummies(
    data,
    prefix=None,
    prefix_sep: str | Iterable[str] | dict[str, str] = "_",
    dummy_na: bool = False,
    columns=None,
    sparse: bool = False,
    drop_first: bool = False,
    dtype: NpDtype | None = None,
) -> DataFrame:
    """
    Convert categorical variable into dummy/indicator variables.

    Each variable is converted in as many 0/1 variables as there are different
    values. Columns in the output are each named after a value; if the input is
    a DataFrame, the name of the original variable is prepended to the value.

    Parameters
    ----------
    data : array-like, Series, or DataFrame
        Data of which to get dummy indicators.
    prefix : str, list 

### 适用于GAN的完整数据

In [164]:
# 对星期进行独热编码
training_demand_encode = pd.get_dummies(training_demand, columns=['week'], prefix='day')

# day_map={      #    0否  1是  
#     'False':0,
#     'True':1}

for i in range(1,8):
    training_demand_encode[f'day_{i}'] = training_demand_encode[f'day_{i}'].astype(int)
    # training_demand_encode[f'day_{i}'] = training_demand_encode[f'day_{i}'].map(day_map)

# 查看编码后的 DataFrame
training_demand_encode

,GridID,date,TSegmentID,month,Demand,temp_h,temp_l,weather,wind,workday,...,poi_count_3,poi_count_4,total_count,day_1,day_2,day_3,day_4,day_5,day_6,day_7
0,448.0,2017-07-19,105,7,172,32.0,25.0,2,2.0,1,...,1773,2758,12499,0,0,1,0,0,0,0
3,448.0,2017-07-07,123,7,161,31.0,25.0,3,3.0,1,...,1773,2758,12499,0,0,0,0,1,0,0
9,448.0,2017-07-09,105,7,153,33.0,26.0,2,3.0,0,...,1773,2758,12499,0,0,0,0,0,0,1
20,448.0,2017-07-15,109,7,147,30.0,25.0,3,4.0,0,...,1773,2758,12499,0,0,0,0,0,1,0
34,447.0,2017-07-19,105,7,143,32.0,25.0,2,2.0,1,...,2437,2280,10996,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975868,412.0,2017-07-03,34,7,1,33.0,26.0,2,1.0,1,...,39,45,360,1,0,0,0,0,0,0
975869,412.0,2017-07-01,34,7,1,35.0,26.0,2,1.0,0,...,39,45,360,0,0,0,0,0,1,0
975882,412.0,2017-07-09,33,7,1,33.0,26.0,2,3.0,0,...,39,45,360,0,0,0,0,0,0,1
975883,412.0,2017-07-08,33,7,1,33.0,26.0,2,3.0,0,...,39,45,360,0,0,0,0,0,1,0


In [165]:
# 删除部分无用列
training_demand_encode = training_demand_encode.drop(columns=['GridID','date','month','workday'])
training_demand_encode

,TSegmentID,Demand,temp_h,temp_l,weather,wind,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count,day_1,day_2,day_3,day_4,day_5,day_6,day_7
0,105,172,32.0,25.0,2,2.0,742,655,6571,1773,2758,12499,0,0,1,0,0,0,0
3,123,161,31.0,25.0,3,3.0,742,655,6571,1773,2758,12499,0,0,0,0,1,0,0
9,105,153,33.0,26.0,2,3.0,742,655,6571,1773,2758,12499,0,0,0,0,0,0,1
20,109,147,30.0,25.0,3,4.0,742,655,6571,1773,2758,12499,0,0,0,0,0,1,0
34,105,143,32.0,25.0,2,2.0,500,607,5172,2437,2280,10996,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975868,34,1,33.0,26.0,2,1.0,3,7,266,39,45,360,1,0,0,0,0,0,0
975869,34,1,35.0,26.0,2,1.0,3,7,266,39,45,360,0,0,0,0,0,1,0
975882,33,1,33.0,26.0,2,3.0,3,7,266,39,45,360,0,0,0,0,0,0,1
975883,33,1,33.0,26.0,2,3.0,3,7,266,39,45,360,0,0,0,0,0,1,0


In [166]:
# 指定要放在第一列的列名
target_column = 'Demand'

# 获取列索引
columns = list(training_demand_encode.columns)
target_index = columns.index(target_column)

# 将目标列插入到第一列
columns.insert(0, columns.pop(target_index))

# 重新构建 DataFrame 的列顺序
training_demand_encode = training_demand_encode[columns]

training_demand_encode

,Demand,TSegmentID,temp_h,temp_l,weather,wind,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,total_count,day_1,day_2,day_3,day_4,day_5,day_6,day_7
0,172,105,32.0,25.0,2,2.0,742,655,6571,1773,2758,12499,0,0,1,0,0,0,0
3,161,123,31.0,25.0,3,3.0,742,655,6571,1773,2758,12499,0,0,0,0,1,0,0
9,153,105,33.0,26.0,2,3.0,742,655,6571,1773,2758,12499,0,0,0,0,0,0,1
20,147,109,30.0,25.0,3,4.0,742,655,6571,1773,2758,12499,0,0,0,0,0,1,0
34,143,105,32.0,25.0,2,2.0,500,607,5172,2437,2280,10996,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975868,1,34,33.0,26.0,2,1.0,3,7,266,39,45,360,1,0,0,0,0,0,0
975869,1,34,35.0,26.0,2,1.0,3,7,266,39,45,360,0,0,0,0,0,1,0
975882,1,33,33.0,26.0,2,3.0,3,7,266,39,45,360,0,0,0,0,0,0,1
975883,1,33,33.0,26.0,2,3.0,3,7,266,39,45,360,0,0,0,0,0,1,0


In [168]:
# 保存数据
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\demand_training_full.csv'
training_demand_encode.to_csv(csv_save, index=False)

# For time GAN

In [24]:
# grid订单数排序
# 使用groupby和sum函数对GridID进行分组求和
sum_demand_by_gridid = demand_tg.groupby('GridID')['Demand'].sum().reset_index()

# 按照求和结果（Demand）从高到低进行排序
sum_demand_by_gridid_sorted = sum_demand_by_gridid.sort_values(by='Demand', ascending=False)

# 打印排序后的结果
print(sum_demand_by_gridid_sorted)

     GridID   Demand
222   448.0  1353289
221   447.0  1124589
238   467.0   803300
186   404.0   726250
204   427.0   700379
..      ...      ...
30    154.0        1
138   349.0        1
28    147.0        1
85    268.0        1
0      25.0        1

[256 rows x 2 columns]


In [48]:
sum_demand_by_gridid_sorted.iloc[30]

GridID      407.0
Demand    27939.0
Name: 189, dtype: float64

In [73]:
demand_tg

,GridID,MappedTime,Demand
0,25.0,2017-10-06 09:10:00,1
1,31.0,2017-05-13 11:10:00,1
2,32.0,2017-05-28 15:10:00,1
3,37.0,2017-06-13 14:10:00,1
4,38.0,2017-05-29 18:20:00,1
...,...,...,...
976002,488.0,2017-10-29 18:50:00,1
976003,488.0,2017-10-29 20:00:00,1
976004,488.0,2017-10-30 20:00:00,1
976005,488.0,2017-10-30 21:10:00,1


In [74]:
df_0 = demand_tg[demand_tg['GridID'] == 81]
df_1 = df_0
df_2 = df_0
df_3 = df_0
df_4 = df_0
df_5 = df_0
df_6 = df_0
df_7 = df_0
df_8 = df_0
df_9 = df_0
df_0

,GridID,MappedTime,Demand


In [129]:
ids = [448,447,467,404,427,405,480,402,482,407]
dfs = [df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9]

for i in range(10):
    # 使用条件筛选提取GridID为特定值的数据
    dfs[i] = demand_tg[demand_tg['GridID'] == ids[i]]

dfs[0]

,GridID,MappedTime,Demand
687622,448.0,2017-04-30 23:20:00,1
687623,448.0,2017-04-30 23:40:00,2
687624,448.0,2017-04-30 23:50:00,20
687625,448.0,2017-05-01 00:00:00,36
687626,448.0,2017-05-01 00:10:00,33
...,...,...,...
714068,448.0,2017-10-31 23:10:00,37
714069,448.0,2017-10-31 23:20:00,52
714070,448.0,2017-10-31 23:30:00,45
714071,448.0,2017-10-31 23:40:00,27


In [124]:
df0 = dfs[0].copy()
df1 = dfs[1].copy()
df2 = dfs[2].copy()
df3 = dfs[3].copy()
df4 = dfs[4].copy()
df5 = dfs[5].copy()
df6 = dfs[6].copy()
df7 = dfs[7].copy()
df8 = dfs[8].copy()
df9 = dfs[9].copy()

df0.rename(columns={'Demand': 'Demand_0'}, inplace=True)
df1.rename(columns={'Demand': 'Demand_1'}, inplace=True)
df2.rename(columns={'Demand': 'Demand_2'}, inplace=True)
df3.rename(columns={'Demand': 'Demand_3'}, inplace=True)
df4.rename(columns={'Demand': 'Demand_4'}, inplace=True)
df5.rename(columns={'Demand': 'Demand_5'}, inplace=True)
df6.rename(columns={'Demand': 'Demand_6'}, inplace=True)
df7.rename(columns={'Demand': 'Demand_7'}, inplace=True)
df8.rename(columns={'Demand': 'Demand_8'}, inplace=True)
df9.rename(columns={'Demand': 'Demand_9'}, inplace=True)

df0 = df0.drop(columns=['GridID'])
df1 = df1.drop(columns=['GridID'])
df2 = df2.drop(columns=['GridID'])
df3 = df3.drop(columns=['GridID'])
df4 = df4.drop(columns=['GridID'])
df5 = df5.drop(columns=['GridID'])
df6 = df6.drop(columns=['GridID'])
df7 = df7.drop(columns=['GridID'])
df8 = df8.drop(columns=['GridID'])
df9 = df9.drop(columns=['GridID'])


df4

,MappedTime,Demand_4
500924,2017-04-30 23:50:00,13
500925,2017-05-01 00:00:00,16
500926,2017-05-01 00:10:00,13
500927,2017-05-01 00:20:00,6
500928,2017-05-01 00:30:00,12
...,...,...
526992,2017-10-31 23:10:00,31
526993,2017-10-31 23:20:00,20
526994,2017-10-31 23:30:00,29
526995,2017-10-31 23:40:00,27


In [123]:
df4.info()

AttributeError: 'NoneType' object has no attribute 'info'

In [100]:
# # 定义起始时间和结束时间
# start_date = '2017-04-30'
# end_date = '2017-10-31'

# # 生成每十分钟为间隔的时间序列
# time_intervals = pd.date_range(start=start_date, end=end_date, freq='10T')

# # 将时间序列转换为 DataFrame
# df_time_intervals = pd.DataFrame({'MappedTime': time_intervals})

# # 显示生成的 DataFrame
# print(df_time_intervals)

               MappedTime
0     2017-04-30 00:00:00
1     2017-04-30 00:10:00
2     2017-04-30 00:20:00
3     2017-04-30 00:30:00
4     2017-04-30 00:40:00
...                   ...
26492 2017-10-30 23:20:00
26493 2017-10-30 23:30:00
26494 2017-10-30 23:40:00
26495 2017-10-30 23:50:00
26496 2017-10-31 00:00:00

[26497 rows x 1 columns]


D:\Temp\ipykernel_88428\2470854852.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_intervals = pd.date_range(start=start_date, end=end_date, freq='10T')


In [128]:
df0

,MappedTime,Demand_0
687622,2017-04-30 23:20:00,1
687623,2017-04-30 23:40:00,2
687624,2017-04-30 23:50:00,20
687625,2017-05-01 00:00:00,36
687626,2017-05-01 00:10:00,33
...,...,...
714068,2017-10-31 23:10:00,37
714069,2017-10-31 23:20:00,52
714070,2017-10-31 23:30:00,45
714071,2017-10-31 23:40:00,27


In [132]:
merged_df = pd.merge(df0, df1, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df2, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df3, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df4, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df5, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df6, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df7, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df8, on='MappedTime', how='left').fillna(0)
merged_df = pd.merge(merged_df, df9, on='MappedTime', how='left').fillna(0)

# 打印合并后的 DataFrame
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26451 entries, 0 to 26450
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   MappedTime  26451 non-null  datetime64[ns]
 1   Demand_0    26451 non-null  int64         
 2   Demand_1    26451 non-null  float64       
 3   Demand_2    26451 non-null  float64       
 4   Demand_3    26451 non-null  float64       
 5   Demand_4    26451 non-null  float64       
 6   Demand_5    26451 non-null  float64       
 7   Demand_6    26451 non-null  float64       
 8   Demand_7    26451 non-null  float64       
 9   Demand_8    26451 non-null  float64       
 10  Demand_9    26451 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 2.2 MB


In [139]:
for i in range(1,10):
    merged_df[f'Demand_{i}'] = merged_df[f'Demand_{i}'].astype('int64')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26451 entries, 0 to 26450
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   MappedTime  26451 non-null  datetime64[ns]
 1   Demand_0    26451 non-null  int64         
 2   Demand_1    26451 non-null  int64         
 3   Demand_2    26451 non-null  int64         
 4   Demand_3    26451 non-null  int64         
 5   Demand_4    26451 non-null  int64         
 6   Demand_5    26451 non-null  int64         
 7   Demand_6    26451 non-null  int64         
 8   Demand_7    26451 non-null  int64         
 9   Demand_8    26451 non-null  int64         
 10  Demand_9    26451 non-null  int64         
dtypes: datetime64[ns](1), int64(10)
memory usage: 2.2 MB


In [140]:
# 筛选6-8三周的数据作为训练集
start_date = '2017-06-01'
end_date = '2017-09-30'

# 使用条件筛选选择日期在指定范围内的数据
training_demand = merged_df[(merged_df['MappedTime'] >= start_date) & (merged_df['MappedTime'] <= end_date)]

# 保存数据
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\for_timeGAN0.csv'
training_demand.to_csv(csv_save, index=False)

In [141]:
del training_demand['MappedTime']
del training_demand['Demand_0']
del training_demand['Demand_1']
del training_demand['Demand_2']
del training_demand['Demand_3']

csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\for_timeGAN1.csv'
training_demand.to_csv(csv_save, index=False)

# 废案

In [154]:
training_demand.columns.to_list()

['Demand', 'TSegmentID', 'week', 'weather', 'total_count']

In [21]:
basic_info(demand['wind'])

(6.0, 0.0, 2.6391051496107307, 1.0207875650728848)

In [22]:
quantiles = spilt_quantile(demand['wind'],3)

In [23]:
quantiles = spilt_quantile(demand['wind'],3)

def demand_level3(demand):
    if demand > quantiles[0.666]:
        return 'High'
    elif demand > quantiles[0.333]:
        return 'Medium'
    else:
        return 'Low'

# 添加需求级别列到 DataFrame
demand['WindLevel'] = demand['wind'].apply(demand_level3)
demand

,date,GridID,TSegmentID,month,Demand,DemandLevel,week,temp_h,temp_l,weather,wind,workday,WindLevel
0,2017-07-22,448.0,17,7,866,Very High,6,29.0,25.0,4,2.0,0,Low
1,2017-09-16,448.0,17,9,824,Very High,6,32.0,26.0,1,3.0,0,Medium
2,2017-10-17,448.0,20,10,814,Very High,2,27.0,24.0,4,2.0,1,Low
3,2017-07-07,448.0,20,7,794,Very High,5,31.0,25.0,3,3.0,1,Medium
4,2017-08-27,448.0,17,8,791,Very High,7,30.0,26.0,4,3.0,0,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245198,2017-05-01,409.0,13,5,1,Very Low,1,32.0,25.0,1,5.0,1,High
245199,2017-10-26,409.0,12,10,1,Very Low,4,21.0,20.4,1,3.5,1,High
245200,2017-10-23,409.0,12,10,1,Very Low,1,27.0,21.0,1,2.0,1,Low
245201,2017-10-22,409.0,12,10,1,Very Low,7,27.0,22.0,1,2.0,0,Low


In [24]:
basic_info(demand['temp_h'])

(37.0, 21.0, 31.61420129443767, 2.9184664019508233)

In [25]:
spilt_quantile(demand['temp_h'],3)

0.333    31.0
0.666    33.0
Name: temp_h, dtype: float64

In [26]:
quantiles = spilt_quantile(demand['temp_h'],3)

# 添加需求级别列到 DataFrame
demand['temp_h_Level'] = demand['temp_h'].apply(demand_level3)
demand

,date,GridID,TSegmentID,month,Demand,DemandLevel,week,temp_h,temp_l,weather,wind,workday,WindLevel,temp_h_Level
0,2017-07-22,448.0,17,7,866,Very High,6,29.0,25.0,4,2.0,0,Low,Low
1,2017-09-16,448.0,17,9,824,Very High,6,32.0,26.0,1,3.0,0,Medium,Medium
2,2017-10-17,448.0,20,10,814,Very High,2,27.0,24.0,4,2.0,1,Low,Low
3,2017-07-07,448.0,20,7,794,Very High,5,31.0,25.0,3,3.0,1,Medium,Low
4,2017-08-27,448.0,17,8,791,Very High,7,30.0,26.0,4,3.0,0,Medium,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245198,2017-05-01,409.0,13,5,1,Very Low,1,32.0,25.0,1,5.0,1,High,Medium
245199,2017-10-26,409.0,12,10,1,Very Low,4,21.0,20.4,1,3.5,1,High,Low
245200,2017-10-23,409.0,12,10,1,Very Low,1,27.0,21.0,1,2.0,1,Low,Low
245201,2017-10-22,409.0,12,10,1,Very Low,7,27.0,22.0,1,2.0,0,Low,Low


In [27]:
basic_info(demand['temp_l'])

(28.0, 19.0, 25.289980954556018, 1.5559706796072659)

In [28]:
spilt_quantile(demand['temp_l'],3)

0.333    25.0
0.666    26.0
Name: temp_l, dtype: float64

In [29]:
quantiles = spilt_quantile(demand['temp_l'],3)

# 添加需求级别列到 DataFrame
demand['temp_l_Level'] = demand['temp_l'].apply(demand_level3)
demand

,date,GridID,TSegmentID,month,Demand,DemandLevel,week,temp_h,temp_l,weather,wind,workday,WindLevel,temp_h_Level,temp_l_Level
0,2017-07-22,448.0,17,7,866,Very High,6,29.0,25.0,4,2.0,0,Low,Low,Low
1,2017-09-16,448.0,17,9,824,Very High,6,32.0,26.0,1,3.0,0,Medium,Medium,Medium
2,2017-10-17,448.0,20,10,814,Very High,2,27.0,24.0,4,2.0,1,Low,Low,Low
3,2017-07-07,448.0,20,7,794,Very High,5,31.0,25.0,3,3.0,1,Medium,Low,Low
4,2017-08-27,448.0,17,8,791,Very High,7,30.0,26.0,4,3.0,0,Medium,Low,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245198,2017-05-01,409.0,13,5,1,Very Low,1,32.0,25.0,1,5.0,1,High,Medium,Low
245199,2017-10-26,409.0,12,10,1,Very Low,4,21.0,20.4,1,3.5,1,High,Low,Low
245200,2017-10-23,409.0,12,10,1,Very Low,1,27.0,21.0,1,2.0,1,Low,Low,Low
245201,2017-10-22,409.0,12,10,1,Very Low,7,27.0,22.0,1,2.0,0,Low,Low,Low


In [30]:
# 删除部分已经离散化的列
demand.drop(columns=['Demand','temp_h','temp_l','wind','workday','date'])

,GridID,TSegmentID,month,DemandLevel,week,weather,WindLevel,temp_h_Level,temp_l_Level
0,448.0,17,7,Very High,6,4,Low,Low,Low
1,448.0,17,9,Very High,6,1,Medium,Medium,Medium
2,448.0,20,10,Very High,2,4,Low,Low,Low
3,448.0,20,7,Very High,5,3,Medium,Low,Low
4,448.0,17,8,Very High,7,4,Medium,Low,Medium
...,...,...,...,...,...,...,...,...,...
245198,409.0,13,5,Very Low,1,1,High,Medium,Low
245199,409.0,12,10,Very Low,4,1,High,Low,Low
245200,409.0,12,10,Very Low,1,1,Low,Low,Low
245201,409.0,12,10,Very Low,7,1,Low,Low,Low


In [13]:
# 查看需求数量出现的频率
# demand_counts = demand['Demand'].value_counts().sort_index()
# demand_counts

Demand
1      45305
2      22713
3      15578
4      11743
5       9371
       ...  
791        1
794        1
814        1
824        1
866        1
Name: count, Length: 714, dtype: int64

In [14]:
# demand['Demand'].std()

84.79110438140734

In [15]:
# demand['Demand'].mean()

45.362494291492695

In [16]:
# 根据需求数量的频率确定断点-分位数 (percentile)离散法
quantiles = demand['Demand'].quantile([0.2, 0.4, 0.6, 0.8])

# 显示划分断点
quantiles

0.2     2.0
0.4     5.0
0.6    14.0
0.8    64.0
Name: Demand, dtype: float64

In [17]:
# 划分需求数量为五类
def demand_level5(demand):
    if demand > quantiles[0.8]:
        return 'Very High'
    elif demand > quantiles[0.6]:
        return 'High'
    elif demand > quantiles[0.4]:
        return 'Medium'
    elif demand > quantiles[0.2]:
        return 'Low'
    else:
        return 'Very Low'

# 添加需求级别列到 DataFrame
demand['DemandLevel'] = demand['Demand'].apply(demand_level5)
demand

,GridID,TSegmentID,month,date,Demand,DemandLevel
182557,448.0,17,7,2017-07-22,866,Very High
182613,448.0,17,9,2017-09-16,824,Very High
183196,448.0,20,10,2017-10-17,814,Very High
183094,448.0,20,7,2017-07-07,794,Very High
182593,448.0,17,8,2017-08-27,791,Very High
...,...,...,...,...,...,...
104418,409.0,13,5,2017-05-01,1,Very Low
104412,409.0,12,10,2017-10-26,1,Very Low
104409,409.0,12,10,2017-10-23,1,Very Low
104408,409.0,12,10,2017-10-22,1,Very Low


### 4.加入POI数据

In [31]:
# 对total_count进行离散化处理
basic_info(df_poi['total_count'])

(13459, 0, 232.0633946830266, 1168.1613521870042)

In [32]:
quantiles = spilt_quantile(df_poi['total_count'],5)
quantiles

0.2     3.0
0.4     8.0
0.6    14.0
0.8    39.4
Name: total_count, dtype: float64

In [33]:
# 添加需求级别列到 DataFrame
df_poi['POI_total'] = df_poi['total_count'].apply(demand_level5)
df_poi

,GridID,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,poi_count_5,poi_count_6,poi_count_7,poi_count_8,...,poi_count_15,poi_count_16,poi_count_17,poi_count_18,poi_count_19,poi_count_20,poi_count_21,poi_count_22,total_count,POI_total
0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0,Very Low
1,1,0.008191,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,1,Very Low
2,2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0,Very Low
3,3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0,Very Low
4,4,0.016383,0.027352,0.010784,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,4,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,484,0.000000,0.054705,0.000000,0.0,0.0,0.0,0.0,0.007859,0.180832,...,0.0,0.09508,0.0,0.0,0.0,0.0,0.0,0.0,12,Medium
485,485,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0,Very Low
486,486,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0,Very Low
487,487,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0,Very Low


In [34]:
# 对poi_count进行离散化处理-
for i in range(23):
    quantiles = spilt_quantile(df_poi[f'poi_count_{i}'],5)
    def percentage_level(percentage):
        if percentage > quantiles[0.8]:
            return 'Very High'
        elif percentage > quantiles[0.6]:
            return 'High'
        elif percentage > quantiles[0.4]:
            return 'Medium'
        elif percentage > quantiles[0.2]:
            return 'Low'
        else:
            return 'Very Low'
    # 添加级别列到 DataFrame
    df_poi[f'poi_Level_{i}'] = df_poi[f'poi_count_{i}'].apply(demand_level5)
df_poi

,GridID,poi_count_0,poi_count_1,poi_count_2,poi_count_3,poi_count_4,poi_count_5,poi_count_6,poi_count_7,poi_count_8,...,poi_Level_13,poi_Level_14,poi_Level_15,poi_Level_16,poi_Level_17,poi_Level_18,poi_Level_19,poi_Level_20,poi_Level_21,poi_Level_22
0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
1,1,0.008191,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
2,2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
3,3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
4,4,0.016383,0.027352,0.010784,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,484,0.000000,0.054705,0.000000,0.0,0.0,0.0,0.0,0.007859,0.180832,...,Very Low,Very High,Very Low,Very High,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
485,485,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
486,486,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
487,487,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low


In [35]:
# 去除原始值
df_poi = df_poi.drop(columns=['total_count'])
for i in range(23):
    df_poi = df_poi.drop(columns=[f'poi_count_{i}'])
df_poi

,GridID,POI_total,poi_Level_0,poi_Level_1,poi_Level_2,poi_Level_3,poi_Level_4,poi_Level_5,poi_Level_6,poi_Level_7,...,poi_Level_13,poi_Level_14,poi_Level_15,poi_Level_16,poi_Level_17,poi_Level_18,poi_Level_19,poi_Level_20,poi_Level_21,poi_Level_22
0,0,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
1,1,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
2,2,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
3,3,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
4,4,Low,Very Low,Medium,Medium,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,484,Medium,Very Low,High,Very Low,Very Low,Very Low,Very Low,Very Low,High,...,Very Low,Very High,Very Low,Very High,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
485,485,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
486,486,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low
487,487,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,...,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low,Very Low


In [36]:
# 连接demand和POI
demand = demand.set_index('GridID').join(df_poi.set_index('GridID'),how='inner')
# 将索引转换为普通的一列
demand = demand.reset_index()
demand

,GridID,date,TSegmentID,month,Demand,DemandLevel,week,temp_h,temp_l,weather,...,poi_Level_13,poi_Level_14,poi_Level_15,poi_Level_16,poi_Level_17,poi_Level_18,poi_Level_19,poi_Level_20,poi_Level_21,poi_Level_22
0,448.0,2017-07-22,17,7,866,Very High,6,29.0,25.0,4,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
1,448.0,2017-09-16,17,9,824,Very High,6,32.0,26.0,1,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
2,448.0,2017-10-17,20,10,814,Very High,2,27.0,24.0,4,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
3,448.0,2017-07-07,20,7,794,Very High,5,31.0,25.0,3,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
4,448.0,2017-08-27,17,8,791,Very High,7,30.0,26.0,4,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245198,409.0,2017-05-01,13,5,1,Very Low,1,32.0,25.0,1,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low
245199,409.0,2017-10-26,12,10,1,Very Low,4,21.0,20.4,1,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low
245200,409.0,2017-10-23,12,10,1,Very Low,1,27.0,21.0,1,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low
245201,409.0,2017-10-22,12,10,1,Very Low,7,27.0,22.0,1,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low


In [37]:
demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245203 entries, 0 to 245202
Data columns (total 39 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   GridID        245203 non-null  float64       
 1   date          245203 non-null  datetime64[ns]
 2   TSegmentID    245203 non-null  int32         
 3   month         245203 non-null  int32         
 4   Demand        245203 non-null  int64         
 5   DemandLevel   245203 non-null  object        
 6   week          245203 non-null  int64         
 7   temp_h        245203 non-null  float64       
 8   temp_l        245203 non-null  float64       
 9   weather       245203 non-null  int64         
 10  wind          245203 non-null  float64       
 11  workday       245203 non-null  int64         
 12  WindLevel     245203 non-null  object        
 13  temp_h_Level  245203 non-null  object        
 14  temp_l_Level  245203 non-null  object        
 15  POI_total     245

In [38]:
demand = demand.drop(columns=['GridID','date','Demand','workday','temp_h','temp_l','wind'])

In [39]:
demand['TSegmentID'] = demand['TSegmentID'].astype(object)
demand['month'] = demand['month'].astype(object)
demand['week'] = demand['week'].astype(object)
demand['weather'] = demand['weather'].astype(object)
demand['weather'] = demand['weather'].astype(object)
demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245203 entries, 0 to 245202
Data columns (total 32 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   TSegmentID    245203 non-null  object
 1   month         245203 non-null  object
 2   DemandLevel   245203 non-null  object
 3   week          245203 non-null  object
 4   weather       245203 non-null  object
 5   WindLevel     245203 non-null  object
 6   temp_h_Level  245203 non-null  object
 7   temp_l_Level  245203 non-null  object
 8   POI_total     245203 non-null  object
 9   poi_Level_0   245203 non-null  object
 10  poi_Level_1   245203 non-null  object
 11  poi_Level_2   245203 non-null  object
 12  poi_Level_3   245203 non-null  object
 13  poi_Level_4   245203 non-null  object
 14  poi_Level_5   245203 non-null  object
 15  poi_Level_6   245203 non-null  object
 16  poi_Level_7   245203 non-null  object
 17  poi_Level_8   245203 non-null  object
 18  poi_Level_9   245203 non

In [40]:
# 获取列索引
columns = demand.columns.tolist()

# 调换列 'B' 和 'C' 的位置
index_a = columns.index('TSegmentID')
index_b = columns.index('month')
index_c = columns.index('DemandLevel')
columns[index_a], columns[index_b], columns[index_c]= columns[index_c], columns[index_a], columns[index_b]

# 根据新的列顺序重建 DataFrame
demand = demand[columns]
demand

,DemandLevel,TSegmentID,month,week,weather,WindLevel,temp_h_Level,temp_l_Level,POI_total,poi_Level_0,...,poi_Level_13,poi_Level_14,poi_Level_15,poi_Level_16,poi_Level_17,poi_Level_18,poi_Level_19,poi_Level_20,poi_Level_21,poi_Level_22
0,Very High,17,7,6,4,Low,Low,Low,Very High,Very High,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
1,Very High,17,9,6,1,Medium,Medium,Medium,Very High,Very High,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
2,Very High,20,10,2,4,Low,Low,Low,Very High,Very High,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
3,Very High,20,7,5,3,Medium,Low,Low,Very High,Very High,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
4,Very High,17,8,7,4,Medium,Low,Medium,Very High,Very High,...,Very High,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very High,Very Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245198,Very Low,13,5,1,1,High,Medium,Low,Very High,Very High,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low
245199,Very Low,12,10,4,1,High,Low,Low,Very High,Very High,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low
245200,Very Low,12,10,1,1,Low,Low,Low,Very High,Very High,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low
245201,Very Low,12,10,7,1,Low,Low,Low,Very High,Very High,...,Very Low,Very High,Very High,Very High,Very High,Very High,Very Low,Very Low,Very Low,Very Low


In [41]:
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\forBN.csv'
# demand.to_csv(csv_save, index=False)

In [42]:
demand['weather'].min(), demand['weather'].max(), 

(1, 5)

In [47]:
mapping_dict_TSegmentID = {
    0: 'night',
    1: 'night',
    2: 'nignt',
    3: 'nignt',
    4: 'nignt',
    5: 'nignt',
    6: 'morning',
    7: 'morning',
    8: 'morning',
    9: 'morning',
    10: 'morning',
    11: 'noon',
    12: 'noon',
    13: 'noon',
    14: 'afternoon',
    15: 'afternoon',
    16: 'afternoon',
    17: 'afternoon',
    18: 'nignt',
    19: 'nignt',
    20: 'nignt',
    21: 'nignt',
    22: 'nignt',
    23: 'nignt',
}

mapping_dict_month = {
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
}

mapping_dict_week = {
    1: 'workday',
    2: 'workday',
    3: 'workday',
    4: 'workday',
    5: 'workday',
    6: 'weekend',
    7: 'weekend',
}

mapping_dict_weather = {
    1: 'best',
    2: 'better',
    3: 'medium',
    4: 'worse',
    5: 'worst',
}


# 使用映射字典进行值的映射
demand['TSegmentID'] = demand['TSegmentID'].map(mapping_dict_TSegmentID)
demand['month'] = demand['month'].map(mapping_dict_month)
demand['week'] = demand['week'].map(mapping_dict_week)
demand['weather'] = demand['weather'].map(mapping_dict_weather)

In [49]:
csv_save = r'D:\系统默认\Documents\gradProject\rawData\data1\forBN1.csv'
demand.to_csv(csv_save, index=False)

In [45]:
# demand